In [9]:
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  #data visualization library
from sklearn.tree import DecisionTreeClassifier  # Decision tree algorithm
from sklearn.metrics import accuracy_score  # evaluation metrics

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [10]:
df = pd.read_csv('data.csv')
df.drop(columns=['url', 'source', 'starts_with_ip', 'has_punycode', 'dash_count', 'tld_count', 'domain_has_digits',
        'nan_char_entropy', 'has_internal_links', 'whois_data'], inplace=True)  # drop unrelevant columns


# Print the first 5 rows of the DataFrame 'df'
df.head(5)

,label,url_length,url_entropy,digit_letter_ratio,dot_count,at_count,subdomain_count,domain_age_days
0,legitimate,45,3.924535,0.0,4,0,3,8250.0
1,legitimate,17,3.572469,0.0,1,0,0,10106.0
2,legitimate,16,3.327820,0.0,1,0,0,8111.0
3,legitimate,16,3.500000,0.0,1,0,0,5542.0
4,legitimate,27,3.930270,0.0,3,0,2,5098.0


In [11]:
# Train the model
df = df.dropna()
X = df.drop(columns=['label'])
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Standardize the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Test the model
X_test = scaler.transform(X_test)
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: {:.2f}'.format(accuracy))

Accuracy: 0.97
